<font color='blue' size=5><b>Комментарий ревьюера</b></font>

<font color='blue'>Привет, Наталья! Меня зовут Павел Григорьев, и я буду проверять этот проект.<br>Моя основная цель - не указать на совершённые тобой ошибки, а поделиться своим опытом и помочь тебе совершенствоваться как профессионалу.<br>Спасибо за проделанную работу! Предлагаю общаться на «ты».</font>
<details>
	<summary><u>Инструкция по организационным моментам (кликабельно)</u>⤵</summary>
<font color='blue'>Я буду использовать различные цвета, чтобы было удобнее воспринимать мои комментарии:</font>


---


<font color='blue'>синий текст - просто текст комментария</font>

<font color='green'>✔️ и зеленый текст - все отлично</font>

<font color='orange'>⚠️ и оранжевый текст - сделано все правильно, однако есть рекомендации, на что стоит обратить внимание</font>

<font color='red'>❌ и красный текст - есть недочеты</font>


</details>    
</br>
<font color='blue'>Пожалуйста, не удаляй мои комментарии в случае возврата работы, так будет проще разобраться, какие были недочеты, а также сразу увидеть исправленное. </font>

<font color='blue'>Ответы на мои комментарии лучше тоже помечать.

Например:</font><font color='purple'><b>Комментарий студента</b></font>

<font color='blue'><b>Давай смотреть, что получилось!</b></font>

<span style="color: blue; font-size: 20px; font-weight: bold;">Комментарий ревьюера 2</span>

<span style="color: blue">Привет еще раз. Спасибо, за исправления. Оформление комментариев по работе сохраняется. Только обозначим, что это вторая итерация.</span>

## Введение

Целью настоящего исследования является разработка алгоритма предсказания снижения покупательской активности клиентов магазина 'В Один Клик' и составление рекоммендаций по сохранению активности пользователей на его основе.

## Загрузка данных

In [ ]:
!pip install shap 

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import math

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder
from sklearn.compose import ColumnTransformer


from sklearn.metrics import roc_auc_score, f1_score

from sklearn.model_selection import RandomizedSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer

import shap


<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Хорошее оформление импортов! \
Импорты собраны в одной ячейке, разделены на функциональные группы пустой строкой.</font>

Прочитаем файлы с данными.

In [ ]:
df_market = pd.read_csv('/datasets/market_file.csv')
df_time = pd.read_csv('/datasets/market_time.csv')
df_money = pd.read_csv('/datasets/market_money.csv')
df_profit = pd.read_csv('/datasets/money.csv', sep=';', decimal=',')

<font color='blue'><b>Комментарий ревьюера: </b></font> ⚠️\
<font color='darkorange'>При загрузке в Пандас разделитель дроби можно указывать аргументом `decimal = ','`, тогда данные загружаются в корректном типе данных. (money)</font>
```text
Обратите внимание: в некоторых файлах разделитель значений — точка с запятой, а разделитель десятичных значений — запятая.  
```

<div class="alert-info">
Исправлено
</div>

<span style="color: blue; font-weight: bold">Комментарий ревьюера 2: </span> ✔️\
<span style="color: green"> 👍</span>

## Предобработка

In [ ]:
df_market.head()

In [ ]:
df_money.head()

In [ ]:
df_profit.head()

In [ ]:
df_time.head()

ВИдно что все импортировалось корректно, проанализируем базовую информацию о таблицах.

In [ ]:
print(df_market.info(),df_time.info(),df_money.info(),df_profit.info())

Даные имеют правильный тип

Посчитаем пропуски

In [ ]:
print(df_market.isna().sum(), df_time.isna().sum(), df_money.isna().sum(), df_profit.isna().sum())

В таблицах отсутствуют пропуски.

Проанализируем уникальные значения строковых переменных

In [ ]:
df_money['Период'].unique()

In [ ]:
df_time['Период'].unique()

In [ ]:
for k in df_market.select_dtypes(include='O').columns:
    print(df_market[k].value_counts())

Исправим ошибку в графах 'период' и 'тип сервиса', а также посмотрим на распределение по значениям категориальных переменных.

In [ ]:
df_time['Период'] = df_time['Период'].apply(lambda x:x.replace('предыдцщий_месяц','предыдущий_месяц' ))

<font color='blue'><b>Комментарий ревьюера : </b></font> ✔️\
<font color='green'> 👍</font>

In [ ]:
df_market['Тип сервиса'] = df_market['Тип сервиса'].apply(lambda x:x.replace('стандартт','стандарт' ))

<font color='blue'><b>Комментарий ревьюера : </b></font> ✔️\
<font color='green'> 👍</font>

In [ ]:
for k in df_market.select_dtypes(include='O').columns:
    sns.barplot(x = df_market[k].value_counts().index, y = df_market[k].value_counts().values).set(title=k)
    plt.xticks(rotation=90)
    plt.show()

Целевой признак распределен слегка неравномерно, так что будем использовать стратификацию при разбиении на выборки.

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️ <br>
<font color='green'>Здорово, что не забываешь про промежуточные выводы.</font>

## Исследовательский анализ данных

Данные в df_money содержат выруку за 3 последних месяца, а в df_time - общее время использования за 2 месяца. Объединим их с датафреймом df_market, также поставим id в качестве индекса.

In [ ]:
df_profit.set_index('id', drop=True, inplace=True)

In [ ]:
df_market.set_index('id', drop=True, inplace=True)

In [ ]:
df_time.pivot_table(values='минут', index='id', columns='Период')

In [ ]:
for x in ['препредыдущий_месяц', 'текущий_месяц', 'предыдущий_месяц']:
    name = 'Выручка_' + x
    df_market[name] = df_money.pivot_table(values='Выручка', index='id', columns='Период')[x]

In [ ]:
for x in ['текущий_месяц', 'предыдущий_месяц']:
    name = 'минуты_' + x
    df_market[name] = df_time.pivot_table(values='минут', index='id', columns='Период')[x]

<font color='blue'><b>Комментарий ревьюера: </b></font> ⚠️\
<font color='darkorange'> Таблицы можно "развернуть вширь" по id  методом pivot_table. Необязательно прописывать каждую колонку.</font>

<div class="alert-info">
Исправлено
</div>

<font color='blue'><b>Комментарий ревьюера 2: </b></font> ✔️\
<font color='green'> Отлично!  Развернули таблицы.</font>

In [ ]:
df_market.head()

Рассмотрим распределение количественных переменных.

In [ ]:
df_market.hist(figsize=(10,10), bins=50);

Распределения выглядя правдоподобно, однако есть выброс в графе выручка_текущий_месяц

In [ ]:
df_market['Выручка_текущий_месяц'].sort_values(ascending=False)

Одно значение на порядок больше всех остальных, заменим его на среднее

<font color='blue'><b>Комментарий ревьюера : </b></font> ✔️\
<font color='green'>Действительно явный выброс.
Можно удалить, а можно и заполнить, например значением предыдущего месяца.</font>

In [ ]:
df_market.loc[215380,'Выручка_текущий_месяц'] = df_market['Выручка_текущий_месяц'].mean()

In [ ]:
df_profit['Прибыль'].hist(bins = 50);

Распределение прибыли близко к нормальному. 

In [ ]:
df_market.describe()

In [ ]:
df_market[(df_market['Выручка_предыдущий_месяц']==0) & (df_market['Выручка_препредыдущий_месяц']==0)]

Видно что некоторые клиенты были неактивны в прошлые месяцы, избавимся от них

In [ ]:
df_market.drop(df_market[(df_market['Выручка_предыдущий_месяц']==0) & (df_market['Выручка_препредыдущий_месяц']==0)].index);

<font color='blue'><b>Комментарий ревьюера: </b></font> ❌\
<font color='red'> ~~Нужно удалить пользователей без покупок хотя бы в одном из трёх месяцев, у них активность менее трёх месяцев. ~~</font>

<div class="alert-info">
Исправлено
</div>

<font color='blue'><b>Комментарий ревьюера 2: </b></font> ✔️\
<font color='green'>Неактивные удалены 👌</font>

## Корреляционный анализ

Проверим входные признаки на мультиколлинеарность, для этого построим матрицу корреляции.

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))         
dataplot = sns.heatmap(df_market.corr(method='spearman'), cmap="YlGnBu", annot=True, ax=ax) 

<font color='blue'><b>Комментарий ревьюера: </b></font> ❌\
<font color='red'>~~Все признаки распределены не нормально, поэтому здесь нужно использовать корреляцию Спирмана или Phik или иные подходы, которые устойчивы к выбросам и ненормальному распределению. Корреляция Пирсона, которая здесь используется по умолчанию, нам не подходит.~~</font>

<div class="alert-info">
Исправлено
</div>

<font color='blue'><b>Комментарий ревьюера 2: </b></font> ✔️\
<font color='green'>Здорово,что применена `phik` для анализа и нелинейных зависимостей</font>

<font color='blue'><b>Комментарий ревьюера: </b></font> ⚠️\
<font color='darkorange'>Можно использовать коэффициент корреляции $\phi_K$ , это передовой способ вычисления коэффициентов корреляции для любых типов признаков. Заодно можно посмотреть корреляции с таргетом. Кроме того, в библиотеке есть визуализация. Стоит учитывать, что этот коэффициент показывает не только линейные связи, но и нелинейные. Более подробно понять вид связи поможет визуализация с помощью диаграмм рассеяния. Ссылка на документацию: https://phik.readthedocs.io/en/latest/
Пример кода:</font>
```python
from phik.report import plot_correlation_matrix
from phik import report

# список интервальных признаков
interval_cols = [...]

# вычисление коэффициентов корреляции phi для датафрейма df
phik_overview = df.phik_matrix(interval_cols=interval_cols)

# визуализация тепловой карты коэффициентов корреляции
plot_correlation_matrix(
    phik_overview.values,
    x_labels=phik_overview.columns,
    y_labels=phik_overview.index,
    title=r"correlation $\phi_K$",
    fontsize_factor=1.5,
    figsize=(15, 12)
)
```

Самая большая корреляция - 0.88 между выручкой за текущий и прошлый месяц, это много, но недостаточно чтобы исключить один из признаков. На всякий случай также проведем VIF анализ.

In [ ]:
X = add_constant(df_market).select_dtypes(exclude=['O'])

VIFs = pd.DataFrame()
VIFs['Variable'] = X.columns
VIFs['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(VIFs)

Все значения достаточно небольшие

<font color='blue'><b>Комментарий ревьюера : </b></font> ✔️\
<font color='green'> 👍</font>

## Тренировка модели

Проведем автоматизированный выбор модели с подбором гиперпараметров, для оценки мы будем использовать метрику f1: в нашей задаче снижение покупательской активности кодируется как TP, цена FP (маркировка стабильного клиента как снижающего активность) для компании ниже чем FN (игнорирование клиента, который вероятно снизит покупательскую активность), к тому же у нас имеется дисбаланс классов - TP меньше чем TN, все это диктует выбор f1.

<span style="color: blue; font-weight: bold">Комментарий ревьюера 2: </span> ✔️\
<span style="color: green"> 👍</span>

In [ ]:
RANDOM_STATE = 42
TEST_SIZE = 0.25

X = df_market.drop(['Покупательская активность'], axis=1)
y = df_market['Покупательская активность']

le = LabelEncoder()
#y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size = TEST_SIZE, 
    random_state = RANDOM_STATE,
    stratify = y)

y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
# Прежний уровень кодируется как 0, Снизилась как 1


ohe_columns = ['Разрешить сообщать','Популярная_категория']
ord_columns = ['Тип сервиса']
num_columns = X_train.select_dtypes(exclude='O').columns



data_preprocessor = ColumnTransformer(
    [('ohe', OneHotEncoder(drop='first', sparse=False), ohe_columns),
     ('ord', OrdinalEncoder(), ord_columns),
     ('num', MinMaxScaler(), num_columns)])

pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))])

param_grid = [
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__max_depth': range(2,5),
        'models__max_features': range(2,5),
        'preprocessor__num': [StandardScaler(), MinMaxScaler()],
        'preprocessor__ord':[OneHotEncoder(drop='first', sparse=False), OrdinalEncoder()]
    },
    
    {
        'models': [KNeighborsClassifier()],
        'models__n_neighbors': range(2,5),
        'preprocessor__num': [StandardScaler(), MinMaxScaler()],
        'preprocessor__ord':[OneHotEncoder(drop='first', sparse=False), OrdinalEncoder()]
    },

    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE, 
            solver='liblinear', 
            penalty='l1'
        )],
        'models__C': range(1,5),
        'preprocessor__num': [StandardScaler(), MinMaxScaler()],
        'preprocessor__ord':[OneHotEncoder(drop='first', sparse=False), OrdinalEncoder()]
    }
    
    ,{
        'models': [SVC(random_state=RANDOM_STATE, kernel='linear', probability=True)],
        'models__C': range(1,5),
        'preprocessor__num': [StandardScaler(), MinMaxScaler()],
        'preprocessor__ord':[OneHotEncoder(drop='first', sparse=False), OrdinalEncoder()]
    }
]

randomized_search = RandomizedSearchCV(
    pipe_final, 
    param_grid, 
    cv=5,
    scoring='f1',
    random_state=RANDOM_STATE,
    n_jobs=-1
)
randomized_search.fit(X_train, y_train)

print('Лучшая модель и её параметры:\n\n', randomized_search.best_estimator_)
print ('Метрика лучшей модели на тренировочной выборке:', randomized_search.best_score_)

# проверьте работу модели на тестовой выборке
# рассчитайте прогноз на тестовых данных
y_test_pred = randomized_search.predict(X_test)
print(f'Метрика f1 на тестовой выборке: {f1_score(y_test, y_test_pred)}')
model = randomized_search.best_estimator_

<font color='blue'><b>Комментарий ревьюера : </b></font> ❌\
<font color='red'> ~~Энкодер можно обучать только на обучающих данных.\
Тестовую выборки только трансформируем.(LabelEncoder())~~ </font>

<div class="alert-info">
Исправлено
</div>

<span style="color: blue; font-weight: bold">Комментарий ревьюера 2: </span> ✔️\
<span style="color: green"> 👍</span>

<font color='blue'><b>Комментарий ревьюера : </b></font> ❌\
<font color='red'> ~~По заданию нужно два Энкодера.\
Можно передавать в параметрах для LR, KNN и SVC - OHE, а для Дерева перебрать OHE и OE,\
a можно засунуть в ord_columns какой-нибудь бинарный признак, например 'тип_сервиса'.~~ </font>
```text
Для кодирования категориальных признаков используйте как минимум два кодировщика.
Напоминаем, что для каждой модели можно подготовить данные с разным кодированием и масштабированием.
```

<div class="alert-info">
Исправлено
</div>

<span style="color: blue; font-weight: bold">Комментарий ревьюера 2: </span> ✔️\
<span style="color: green"> 👍</span>

<font color='blue'><b>Комментарий ревьюера : </b></font> ❌\
<font color='red'> ~~По заданию нужно обучить четыре модели~~
```markdown
Обучите четыре модели: KNeighborsClassifier(), DecisionTreeClassifier(), LogisticRegression() и  SVC()
```
~~Есть специальные методы для уменьшения размерности данных в shap.~~</font>

<div class="alert-info">
Исправлено, SVC с линейным ядром работает
</div>

<span style="color: blue; font-weight: bold">Комментарий ревьюера 2: </span> ✔️\
<span style="color: green"> 👍</span>

<font color='blue'><b>Комментарий ревьюера: </b></font> ❌\
<font color='red'> ~~В SVC нужно передать аргумент probability= True. Что бы можно было работать с вероятностями, если модель выиграет.~~</font>

<div class="alert-info">
Исправлено
</div>

<span style="color: blue; font-weight: bold">Комментарий ревьюера 2: </span> ✔️\
<span style="color: green"> 👍</span>

<font color='blue'><b>Комментарий ревьюера: </b></font> ❌\
<font color='red'> ~~Нужно аргументировать выбор метрики.~~</font>

<div class="alert-info">
Исправлено, аргументация в начале секции
</div>

<font color='blue'><b>Комментарий ревьюера: </b></font> ❌\
<font color='red'>~~roc_auc нужно считать по вероятностям классов~~</font>
```python
probabilities = model.predict_proba(features)
probabilities_one = probabilities[:, 1]
print('Площадь ROC-кривой:', roc_auc_score(target, probabilities_one))
```
<font color='blue'>Самое подробное объяснение метрики, что я видел на русском языке, можно посмотреть здесь: https://alexanderdyakonov.wordpress.com/2017/07/28/auc-roc-площадь-под-кривой-ошибок \
А внутри кросс-валидации мы просто указываем метрику ROC_AUC и там никаких вероятностей нам не нужно. Так как эта метрика не дифференцируема на всем промежутке (кажется, разрыв в нуле), то там разработчики кросс-валидации хитро меняют ее на эквивалент Gini, так как Gini = 2 ×AUC_ROC – 1 (можно почитать по ссылке выше)
</font>

<div class="alert-info">
Метрика заменена на f1
</div>

<span style="color: blue; font-weight: bold">Комментарий ревьюера 2: </span> ✔️\
<span style="color: green"> 👍</span>

## Интерпретация модели

Используем SHAP для анализа признаков.

In [ ]:
model.named_steps['preprocessor'].transformers_[0][1].get_feature_names()

In [ ]:
model.named_steps['preprocessor'].transformers_[1][1].get_feature_names()

In [ ]:
model.fit(X_train, y_train)
feature_names = list(model.named_steps['preprocessor'].transformers_[0][1].get_feature_names()) + list(model.named_steps['preprocessor'].transformers_[1][1].get_feature_names()) +list(num_columns)
X_train_data = model.named_steps['preprocessor'].transform(X_train)
X_test_data = model.named_steps['preprocessor'].transform(X_test)
X_train_transformed = pd.DataFrame(data = X_train_data, columns = feature_names)
X_test_transformed = pd.DataFrame(data = X_test_data, columns = feature_names)

<font color='blue'><b>Комментарий ревьюера: </b></font> ⚠️\
<font color='darkorange'> У нас уже есть обученные препроцессор и классификатор в `randomized_search.best_estimator_`</font>

<div class="alert-info">
Если написать что-то вроде 
explainer = shap.LinearExplainer(model, X_train)
shap_values = explainer(X_test)
shap.plots.beeswarm(shap_values, max_display=20) 
то получается ошибка, вроде как на вход explainer'а нужно подавать только estimator, а не весь pipeline, поэтому я отдельно трансформирую данные в начале.
</div>

<span style="color: blue; font-weight: bold">Комментарий ревьюера 2: </span> ⚠️\
<span style="color: blue"> Всё верно, Нужно подавать предобработанные данные, комментарий к тому что эти модели заново обучались.\
Mожно подавать Пайплайн в shap https://github.com/shap/shap/issues/1362</span>

In [ ]:
shap.initjs()

explainer = shap.LinearExplainer(model[1], X_train_transformed)
shap_values = explainer(X_test_transformed)
shap.plots.beeswarm(shap_values, max_display=20) 

In [ ]:
X_test.columns

In [ ]:
X_test['Выручка_препредыдущий_месяц'].hist(bins=50)

Напомним что целевой признак это вероятность снижения покупательской активности, т.е. нам нужно искать признаки, которые понижают целевой признак. Многие из признаков, сильно влияющих на целевой, весьма предсказуемы:
* видно что снижение времени просмотра, числа просмотренных страниц и категорий положительно коррелируеи с вероятностью снижения покупательской активности. Также, ожидаемо, рост числа неоплаченных товаров приводит к снижению покупательской активности. 
* Несколько более интересна зависимость от числа акционных покупок - рост доли акционных покупок приводит к росту вероятности снижения покупательской активности, однако есть обособленная группа клиентов с очень выскокой долей акционных покупок и соответственно очень высокой вероятностью снижения активности, можно предположить что эти люди не являются постоянными пользователями сервиса и приобретают товары исключительно по акциям, что объясняет их нерегулярную активность. 
* Рост числа маркетинговых коммуникаций приводит к снижению целевого признака, однако только на промежутке 6 месяцев, рост числа маркетинговых коммуникаций за последний месяц почти не влияет на целевой признак. Возможно, требуется время чтобы маркетинговые интервенции возымели эффект.
* Общая продолжительность пользования сервисом повышает целевой признак, т.е. более старые пользователи более склонны к снижению покупательской активности чем более новые, хотя зависимость относительно слабая. Возможно исправить это поможет бонусная программа для пользователей-ветеранов.
* Парадоксальным образом рост числа ошибок сервиса приводит к снижению целевого признака, однако малая значимость этого параметра позволяет предположить, что это ошибка модели - пользователи, проводящие больше времени на сайте, также чаще испытывают ошибки, возможно модель недостаточно учла этот факт.
* Наконец, покупка товаров категории бытовой электроники и техники для красоты и здоровья снижает целевой признак, остальные категории слабо влияют на этот показатель. 
* Все остальные категории слабо влияют на целевой признак. В частности, отправка дополнительной информации по товару не влияет на поргноз активности пользователя, так что это не очень эффективный способ привлечения клиентов. Также не влияют и уровень сервиса, и разсер выручки; это означает что отток клиентов мало зависит от их уровня доходов.

<font color='blue'><b>Комментарий ревьюера : </b></font> ✔️\
<font color='green'> 👍</font>

## Сегментация покупателей

Наша цель - увеличить прибыли заказчика. Для этого необходимо снизить отток клиентов, приносящих максимальную прибыль. Здесь мы выделим категорию клиентов с высокой прибылью и высокой вероятностью снижения активности и поанализируем отличие этой категории от общей выборки.

Добавим к тестовой выборке столбцы с прибылью и предсказанной вероятностью снижения активности и построим по ним диаграмму рассеяния.

In [ ]:
X = df_market.drop('Покупательская активность',axis=1)

In [ ]:
model.predict_proba(X)[:,1]

In [ ]:
X['Вероятность_снижения_активности'] = model.predict_proba(X)[:,1]

In [ ]:
X['Прибыль'] = df_profit['Прибыль']

<font color='blue'><b>Комментарий ревьюера: </b></font> ⚠️\
<font color='darkorange'>Сегментацию следует провести на полном Датафрейме. В тестовой выборке слишком мало примеров.</font>

<div class="alert-info">
Исправлено
</div>

<span style="color: blue; font-weight: bold">Комментарий ревьюера 2: </span> ✔️\
<span style="color: green"> 👍</span>

In [ ]:
X.plot.scatter('Прибыль', 'Вероятность_снижения_активности', figsize=(10,10));

По диаграмме видно, что большинство клиентов не будут снижать активность, а также что снижение активности в целом слабо зависит от прибыли. Введем новую переменную риск, равную произведению прибыли на вероятность снижения покупательской активности, так что клиенты с высоким ее значением будут наиболее проблемными для компании.

In [ ]:
X['риск'] = X['Вероятность_снижения_активности']*X['Прибыль']

Рассмотрим гистограмму значений этого параметра.

In [ ]:
X['риск'].hist(bins=50)

Разделим датасет на высокорисковую и низкорисковую часть и поанализируем равспределение параметров в этих группах.

In [ ]:
X_high_risk = X[X['риск']>=2]
X_low_risk = X[X['риск']<2]

<font color='blue'><b>Комментарий ревьюера : </b></font> ✔️\
<font color='green'> 👍</font>

In [ ]:
X_high_risk['Маркет_актив_6_мес'].hist(bins=50)
X_low_risk['Маркет_актив_6_мес'].hist(bins=50, alpha=0.5)
plt.legend(['высокий риск','низкий риск'])
plt.vlines(X_high_risk['Маркет_актив_6_мес'].mean(), 0, 25, color='blue')
plt.vlines(X_low_risk['Маркет_актив_6_мес'].mean(), 0, 25, color='yellow')
plt.title('Маркетологическая акттивность за полгода');

Как мы и предполагали, маркетологическая активность снижает риск потери прибыли.

In [ ]:
X_high_risk['Длительность'].hist(bins=50)
X_low_risk['Длительность'].hist(bins=50, alpha=0.5)
plt.legend(['высокий риск','низкий риск'])
plt.vlines(X_high_risk['Длительность'].mean(), 0, 25, color='blue')
plt.vlines(X_low_risk['Длительность'].mean(), 0, 25, color='yellow')
plt.title('Общая продолжительность регистрации');

Также видно что более старые клиенты с большей вероятностью снизят прибыль для компании.

Теперь проанализируем зависимость от доли акционных покупок. Ранее мы уже отмечали неравномерное распределение доли акционных покупок, можно предположить что ненадежные клиенты, покупающие товары в основном по акциям, приносят низкую прибыль, так что поддержание их численности все равно не очень важно. 

In [ ]:
X_high_risk['Акционные_покупки'].hist(bins=50)
X_low_risk['Акционные_покупки'].hist(bins=50, alpha=0.5)
plt.legend(['высокий риск','низкий риск'])
plt.vlines(X_high_risk['Акционные_покупки'].mean(), 0, 25, color='blue')
plt.vlines(X_low_risk['Акционные_покупки'].mean(), 0, 25, color='yellow')
plt.title('Доля акционных товаров');

Как видно из графика, клиенты с высоеим риском потери прибыли также в среднем покупают больше акционных товаров, так что наше предположение не подтверждается. Таким образом, возможно имеет смысл расширить работу с этой категорией клиентов.

In [ ]:
dat = pd.DataFrame({'Высокий_риск':(X_high_risk['Популярная_категория'].value_counts()/X_high_risk['Популярная_категория'].count()).values,
             'Низкий_риск':(X_low_risk['Популярная_категория'].value_counts()/X_low_risk['Популярная_категория'].count()).values},
            index=X_high_risk['Популярная_категория'].value_counts().index)
dat.plot.bar()
plt.title('Популярная категория');

Как и предполагалось, наиболее 'стабильными' категориями являются техника для красоты и здоровья и мелкая бытовая техника. С другой стороны, наибольшая разница между высокорисковыми и низкорисковыми клиентами в пользу первых наблюдается в товарах для детей и домашнем текстиле и кухонной посуде, можно порекомендовать дополнительные акции для покупателей этой категории товаров. 

<font color='blue'><b>Комментарий ревьюера : </b></font> ✔️\
<font color='green'> 👍</font>

## Заключение

Мы провели анализ данных интернет магазина В Один Клик с целью предсказать вероятность снижения покупательской активности клиентов и составления рекомендаций по сохранению клиентов. Для этого мы провели подбор модели и гиперпараметров с целью оптимизации метрики ROC-AUC, оптимальной моделью оказалась модель логистической регрессии, для интерпретации модели использовалась библиотека SHAP.
По результатам анализа мы можем сделать следующие рекомендации:
* Лучшими предикторами снижения покупательской активности являются снижения времени поведенного на сайте, числа посмотренных страниц и категорий. Клиентам со снижающимися показателями посещаемости можно предоставлять бонусные предложения для избежания их потери.
* Риск снижения покупательской активности растет с долей акционных покупок, при этом клиентоы, покупающие товары только по акции, также приносят значительную прибыль. Можно порекомендовать активизировать работу с этой группой клиентов.  
* Рост числа маркетинговых коммуникаций приводит к снижению риска потери клиента, однако положительный эффект от маркетинговых коммуникаций наступает позднее чем через месяц от их начала. 
* Пользователи, пользовавшиеся сервисом долгое время, с большей вероятностей снизят покупательскую активность чем новые пользователи. Можно порекомендовать расширение бонусных программ для старых пользователей.
* Наименьший риск снижения прибыли среди пользователей предпочитающий технику для красоты и здоровья и мелкую бытовую техника, риск слегка повышен среди покупателей товаров для детей, домашнего текстиля и кухонной посуды, можно рекоммендовать бонусные программы для этих групп пользователей. 

<font color='blue'><b>Комментарий ревьюера : </b></font> ✔️\
<font color='green'> Очень приятно видеть вывод в конце проекта!\
Приведены ответы на главные вопросы проекта.</font>

<font color='blue'><b>Итоговый комментарий ревьюера</b></font>\
<font color='green'>Наталья, хороший проект получился!
Большое спасибо за проделанную работу. Видно, что приложено много усилий.
Выводы и рассуждения получились содержательными, их было интересно читать.
</font>

<font color='steelblue'>Над проектом ещё стоит поработать - есть рекомендации по дополнению некоторых твоих шагов проекта. Такие рекомендации я отметил жёлтыми комментариями. Будет здорово, если ты учтёшь их - так проект станет структурно и содержательно более совершенным.

Также в работе есть критические замечания. К этим замечаниям я оставил пояснительные комментарии красного цвета, в которых перечислил возможные варианты дальнейших действий. Уверен, ты быстро с этим управишься:)

Если о том, что нужно сделать в рамках комментариев, будут возникать вопросы - оставь их, пожалуйста, в комментариях, и я отвечу на них во время следующего ревью.

Также буду рад ответить на любые твои вопросы по проекту или на какие-либо другие, если они у тебя имеются - оставь их в комментариях, и я постараюсь ответить:)</font>

<font color='blue'><b>Жду твой проект на повторном ревью. До встречи :) </b></font>


<font color='blue'><b>Итоговый комментарий ревьюера 2</b></font>\
<font color='green'> Наталья, проект принят! \
Все этапы пройдены. Все рекомендации учтены.\
Надеюсь, тебе понравился процесс выполнения и результат.</font> \
<font color='blue'><b>Спасибо, удачи в освоении профессии!</b></font>